In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ximenml0613/raw/master/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ximenml0613/raw/master/poem_test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
train_df

In [ ]:
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df

In [12]:
# Series.replace(字典)
# {"李白":0, "杜甫":1}
# Series.unique()/Series.value_counts()
u = train_df["作者"].unique()
trans = {n:i for i, n in enumerate(u)}
trans_r = {i:n for i, n in enumerate(u)}
print(trans)
print(trans_r)
y_train = train_df["作者"].replace(trans)
y_test = test_df["作者"].replace(trans)

{'李白': 0, '杜甫': 1, '白居易': 2}
{0: '李白', 1: '杜甫', 2: '白居易'}
